Устанавливаем flair

In [ ]:
!pip install flair

Загружаем тестовый датасет

In [ ]:
# from google.colab import drive
# drive.mount('gdrive')

# %cd /content/gdrive/My Drive/

/content/gdrive/My Drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv("ner_dataset.csv", encoding="latin1")
data = data.fillna(method="ffill")

Класс собирает слова из датасета в предложения

In [ ]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

Собираем корпус из предложений

In [ ]:
getter = SentenceGetter(data)
sentences = getter.sentences
texts = []
for sent in sentences:
    text = ' '.join([str(s[0]) for s in sent])
    texts.append(text)

Функции для отображения лейблов в предложении - каждому предложению соответствует список лейблов

In [ ]:
def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

Собираем датафрейм: text - текст предожения, labels - список лейблов

In [ ]:
df = pd.DataFrame(data=texts, columns=['text'])
y = [sent2labels(s) for s in sentences]
df['labels'] = y

Прогоняем предложения через flair: flair_sent_full для 100 примеров с полной разметкой, flair_sent_labels - для теста (только лейблы)

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger
import re

In [ ]:
tagger = SequenceTagger.load('ner')

In [ ]:
def flair_sent_full(text):
  sentence = Sentence(text)
  tagger.predict(sentence)
  sd = sentence.to_dict(tag_type='ner')
  ners = []
  for entity in sd['entities']:
      ners.append(entity)
  return ners

In [ ]:
def flair_sent_labels(text):
  sentence = Sentence(text)
  tagger.predict(sentence)
  sd = sentence.to_dict(tag_type='ner')
  ners = []
  for entity in sd['entities']:
    ner = entity['labels'][0]
    r = re.search('[A-Z]+', str(entity['labels'][0]))
    if r:
      ners.append(r.group())
  return ners

Датафрейм для 100 примеров: нам нужна полная информация о разметке flair для последующего анализа

In [ ]:
df1 = df.iloc[:100]

In [ ]:
df1['flair'] = df1.apply(lambda row: flair_sent_full(row['text']), axis=1)

In [ ]:
df1.to_pickle("./100_full.pkl")

Полный датафрейм: 5к размеченных предложений

In [ ]:
df2 = df.iloc[:5000]

In [ ]:
df2['flair'] = df2.apply(lambda row: flair_sent_labels(row['text']), axis=1)

In [ ]:
df2.to_pickle("./with_flair.pkl")

Приведем разметку к одному виду - flair

In [ ]:
def change_labels(labels):
  new = []
  for label in labels:
    if label == 'B-per':
      new.append('PER')
    if label in ['B-geo', 'B-gpe']:
      new.append('LOC')
    if label == 'B-org':
      new.append('ORG')
    if label in ['B-art', 'B-eve', 'B-nat']:
      new.append('MISC')
  return new

In [ ]:
df3 = pd.read_pickle("./with_flair.pkl")

In [ ]:
df3['labels_flaired'] = df3.apply(lambda row: change_labels(row['labels']), axis=1)

Сравним качество

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
y_true = MultiLabelBinarizer().fit_transform(df3['labels_flaired'])
y_pred = MultiLabelBinarizer().fit_transform(df3['flair'])

In [ ]:
print(classification_report(y_true, y_pred, target_names=['LOC', 'MISC', 'ORG', 'PER']))

              precision    recall  f1-score   support

         LOC       0.92      0.82      0.86      3108
        MISC       0.04      0.73      0.07        92
         ORG       0.85      0.62      0.71      1695
         PER       0.88      0.92      0.90      1363

   micro avg       0.67      0.78      0.72      6258
   macro avg       0.67      0.77      0.64      6258
weighted avg       0.88      0.78      0.82      6258
 samples avg       0.56      0.62      0.57      6258



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
df3.to_pickle("./final_5k.pkl")